# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [21]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [22]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_id_text = []
    target_id_text = []
    for sentence in source_text.split('\n'):
        sentence_list = []
        for source_word in sentence.split():
            sentence_list.append(source_vocab_to_int[source_word])
        source_id_text.append(sentence_list)
    for sentence in target_text.split('\n'):
        sentence_list = []
        for target_word in sentence.split():
            sentence_list.append(target_vocab_to_int[target_word])
            if target_word == ".":
                sentence_list.append(target_vocab_to_int['<EOS>'])

        target_id_text.append(sentence_list)      
    return (source_id_text, target_id_text)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [24]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [25]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/Users/mmoran/anaconda/envs/word2vec/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [26]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None])
    learning_rate = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    return input, targets, learning_rate, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [27]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [28]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    def lstm_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return enc_cell
    
    enc_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size) for _ in range(num_layers)])
    with_dropout = tf.contrib.rnn.DropoutWrapper(enc_cell, input_keep_prob=keep_prob, output_keep_prob= keep_prob)
    
    rnn_output, rnn_state = tf.nn.dynamic_rnn(with_dropout, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return rnn_output, rnn_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [29]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input, sequence_length=target_sequence_length, time_major=False)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(tf.contrib.rnn.DropoutWrapper(dec_cell, input_keep_prob= keep_prob, output_keep_prob= keep_prob), training_helper, encoder_state, output_layer)
    decoder_output = tf.contrib.seq2seq.dynamic_decode(basic_decoder, impute_finished=True, maximum_iterations=max_summary_length)
    output = tf.contrib.seq2seq.BasicDecoderOutput(decoder_output[0], decoder_output[1])
    return output.rnn_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [30]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function

    start_tokens = tf.tile([tf.constant(start_of_sequence_id, dtype=tf.int32)], [batch_size], name='start_tokens')

    # Helper for the inference process.
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                            start_tokens,
                                                            end_of_sequence_id)
    # Basic decoder
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(tf.contrib.rnn.DropoutWrapper(dec_cell, input_keep_prob= keep_prob, output_keep_prob= keep_prob),
                                                    inference_helper,
                                                    encoder_state,
                                                    output_layer)

    # Perform dynamic decoding using the decoder
    inference_decoder_output = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                        impute_finished=True,
                                                        maximum_iterations=max_target_sequence_length)
    final_output = tf.contrib.seq2seq.BasicDecoderOutput(inference_decoder_output[0], inference_decoder_output[1])

    return final_output.rnn_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [31]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    # 1. Decoder Embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    #2. Make LSTM cell and apply dropout
    def lstm_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size) for _ in range(num_layers)])
    with_dropout = tf.contrib.rnn.DropoutWrapper(dec_cell, input_keep_prob=keep_prob, output_keep_prob= keep_prob)

    # 3. Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    with tf.variable_scope("decode"):
        training_logits = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)
        training_output = tf.contrib.seq2seq.BasicDecoderOutput(training_logits[0], training_logits[1])
    
    with tf.variable_scope("decode", reuse= True):
        inference_logits = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], max_target_sequence_length, target_vocab_size, output_layer, batch_size, keep_prob)
        inference_output = tf.contrib.seq2seq.BasicDecoderOutput(inference_logits[0], inference_logits[1])

    return training_output, inference_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [32]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    encoded = encoding_layer(input_data, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, enc_embedding_size)
    processed = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    decoded = decoding_layer(processed, encoded[1], target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)
    return decoded[0], decoded[1]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [33]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 64
# RNN Size
rnn_size = 512
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.7
display_step = 10

### Build the Graph
Build the graph using the neural network you implemented.

In [34]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [35]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [14]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   10/2154 - Train Accuracy: 0.3414, Validation Accuracy: 0.3956, Loss: 3.2944
Epoch   0 Batch   20/2154 - Train Accuracy: 0.4180, Validation Accuracy: 0.4268, Loss: 2.6913
Epoch   0 Batch   30/2154 - Train Accuracy: 0.4055, Validation Accuracy: 0.4503, Loss: 2.4752
Epoch   0 Batch   40/2154 - Train Accuracy: 0.3972, Validation Accuracy: 0.4702, Loss: 2.2774
Epoch   0 Batch   50/2154 - Train Accuracy: 0.4359, Validation Accuracy: 0.4922, Loss: 2.1324
Epoch   0 Batch   60/2154 - Train Accuracy: 0.4039, Validation Accuracy: 0.4389, Loss: 1.9411
Epoch   0 Batch   70/2154 - Train Accuracy: 0.3890, Validation Accuracy: 0.4283, Loss: 1.8867
Epoch   0 Batch   80/2154 - Train Accuracy: 0.4164, Validation Accuracy: 0.4013, Loss: 1.7963
Epoch   0 Batch   90/2154 - Train Accuracy: 0.3791, Validation Accuracy: 0.4382, Loss: 1.6207
Epoch   0 Batch  100/2154 - Train Accuracy: 0.4301, Validation Accuracy: 0.4496, Loss: 1.6397
Epoch   0 Batch  110/2154 - Train Accuracy: 0.4258, Validati

Epoch   0 Batch  890/2154 - Train Accuracy: 0.6735, Validation Accuracy: 0.6712, Loss: 0.4150
Epoch   0 Batch  900/2154 - Train Accuracy: 0.7410, Validation Accuracy: 0.6555, Loss: 0.4403
Epoch   0 Batch  910/2154 - Train Accuracy: 0.6916, Validation Accuracy: 0.6761, Loss: 0.4141
Epoch   0 Batch  920/2154 - Train Accuracy: 0.7351, Validation Accuracy: 0.7330, Loss: 0.3425
Epoch   0 Batch  930/2154 - Train Accuracy: 0.7125, Validation Accuracy: 0.6818, Loss: 0.3417
Epoch   0 Batch  940/2154 - Train Accuracy: 0.7664, Validation Accuracy: 0.7500, Loss: 0.3642
Epoch   0 Batch  950/2154 - Train Accuracy: 0.7138, Validation Accuracy: 0.7287, Loss: 0.3903
Epoch   0 Batch  960/2154 - Train Accuracy: 0.7680, Validation Accuracy: 0.7266, Loss: 0.3467
Epoch   0 Batch  970/2154 - Train Accuracy: 0.7578, Validation Accuracy: 0.7216, Loss: 0.3406
Epoch   0 Batch  980/2154 - Train Accuracy: 0.7586, Validation Accuracy: 0.7450, Loss: 0.3237
Epoch   0 Batch  990/2154 - Train Accuracy: 0.7594, Validati

Epoch   0 Batch 1770/2154 - Train Accuracy: 0.8961, Validation Accuracy: 0.9041, Loss: 0.0944
Epoch   0 Batch 1780/2154 - Train Accuracy: 0.9375, Validation Accuracy: 0.8537, Loss: 0.0752
Epoch   0 Batch 1790/2154 - Train Accuracy: 0.9129, Validation Accuracy: 0.9219, Loss: 0.0880
Epoch   0 Batch 1800/2154 - Train Accuracy: 0.9484, Validation Accuracy: 0.9368, Loss: 0.0814
Epoch   0 Batch 1810/2154 - Train Accuracy: 0.8734, Validation Accuracy: 0.9176, Loss: 0.0777
Epoch   0 Batch 1820/2154 - Train Accuracy: 0.9391, Validation Accuracy: 0.9389, Loss: 0.0617
Epoch   0 Batch 1830/2154 - Train Accuracy: 0.9199, Validation Accuracy: 0.9332, Loss: 0.1267
Epoch   0 Batch 1840/2154 - Train Accuracy: 0.9358, Validation Accuracy: 0.9176, Loss: 0.0564
Epoch   0 Batch 1850/2154 - Train Accuracy: 0.9564, Validation Accuracy: 0.9332, Loss: 0.1112
Epoch   0 Batch 1860/2154 - Train Accuracy: 0.9605, Validation Accuracy: 0.9034, Loss: 0.0704
Epoch   0 Batch 1870/2154 - Train Accuracy: 0.9055, Validati

Epoch   1 Batch  500/2154 - Train Accuracy: 0.9910, Validation Accuracy: 0.9560, Loss: 0.0408
Epoch   1 Batch  510/2154 - Train Accuracy: 0.8898, Validation Accuracy: 0.9297, Loss: 0.0462
Epoch   1 Batch  520/2154 - Train Accuracy: 0.9414, Validation Accuracy: 0.9396, Loss: 0.0397
Epoch   1 Batch  530/2154 - Train Accuracy: 0.9688, Validation Accuracy: 0.9702, Loss: 0.0422
Epoch   1 Batch  540/2154 - Train Accuracy: 0.9688, Validation Accuracy: 0.9226, Loss: 0.0462
Epoch   1 Batch  550/2154 - Train Accuracy: 0.9315, Validation Accuracy: 0.9155, Loss: 0.0448
Epoch   1 Batch  560/2154 - Train Accuracy: 0.9711, Validation Accuracy: 0.8920, Loss: 0.0381
Epoch   1 Batch  570/2154 - Train Accuracy: 0.9133, Validation Accuracy: 0.9290, Loss: 0.0694
Epoch   1 Batch  580/2154 - Train Accuracy: 0.9424, Validation Accuracy: 0.9162, Loss: 0.0441
Epoch   1 Batch  590/2154 - Train Accuracy: 0.9403, Validation Accuracy: 0.9169, Loss: 0.0439
Epoch   1 Batch  600/2154 - Train Accuracy: 0.9622, Validati

Epoch   1 Batch 1380/2154 - Train Accuracy: 0.9719, Validation Accuracy: 0.9773, Loss: 0.0306
Epoch   1 Batch 1390/2154 - Train Accuracy: 0.9621, Validation Accuracy: 0.9588, Loss: 0.0372
Epoch   1 Batch 1400/2154 - Train Accuracy: 0.9696, Validation Accuracy: 0.9688, Loss: 0.0320
Epoch   1 Batch 1410/2154 - Train Accuracy: 0.9695, Validation Accuracy: 0.9602, Loss: 0.0377
Epoch   1 Batch 1420/2154 - Train Accuracy: 0.9638, Validation Accuracy: 0.9290, Loss: 0.0397
Epoch   1 Batch 1430/2154 - Train Accuracy: 0.9382, Validation Accuracy: 0.9389, Loss: 0.0312
Epoch   1 Batch 1440/2154 - Train Accuracy: 0.9484, Validation Accuracy: 0.9702, Loss: 0.0515
Epoch   1 Batch 1450/2154 - Train Accuracy: 0.9211, Validation Accuracy: 0.9354, Loss: 0.0406
Epoch   1 Batch 1460/2154 - Train Accuracy: 0.9539, Validation Accuracy: 0.9560, Loss: 0.0406
Epoch   1 Batch 1470/2154 - Train Accuracy: 0.9572, Validation Accuracy: 0.9411, Loss: 0.0344
Epoch   1 Batch 1480/2154 - Train Accuracy: 0.9719, Validati

...

Epoch   8 Batch  410/2154 - Train Accuracy: 0.9898, Validation Accuracy: 1.0000, Loss: 0.0076
Epoch   8 Batch  420/2154 - Train Accuracy: 0.9963, Validation Accuracy: 1.0000, Loss: 0.0069
Epoch   8 Batch  430/2154 - Train Accuracy: 0.9934, Validation Accuracy: 0.9964, Loss: 0.0065
Epoch   8 Batch  440/2154 - Train Accuracy: 1.0000, Validation Accuracy: 1.0000, Loss: 0.0043
Epoch   8 Batch  450/2154 - Train Accuracy: 0.9922, Validation Accuracy: 1.0000, Loss: 0.0055
Epoch   8 Batch  460/2154 - Train Accuracy: 0.9891, Validation Accuracy: 1.0000, Loss: 0.0050
Epoch   8 Batch  470/2154 - Train Accuracy: 0.9878, Validation Accuracy: 1.0000, Loss: 0.0060
Epoch   8 Batch  480/2154 - Train Accuracy: 1.0000, Validation Accuracy: 1.0000, Loss: 0.0025
Epoch   8 Batch  490/2154 - Train Accuracy: 0.9977, Validation Accuracy: 1.0000, Loss: 0.0074
Epoch   8 Batch  500/2154 - Train Accuracy: 0.9803, Validation Accuracy: 1.0000, Loss: 0.0132
Epoch   8 Batch  510/2154 - Train Accuracy: 0.9820, Validati

Epoch   8 Batch 1290/2154 - Train Accuracy: 1.0000, Validation Accuracy: 0.9993, Loss: 0.0035
Epoch   8 Batch 1300/2154 - Train Accuracy: 0.9992, Validation Accuracy: 0.9993, Loss: 0.0065
Epoch   8 Batch 1310/2154 - Train Accuracy: 0.9977, Validation Accuracy: 0.9943, Loss: 0.0054
Epoch   8 Batch 1320/2154 - Train Accuracy: 1.0000, Validation Accuracy: 0.9993, Loss: 0.0019
Epoch   8 Batch 1330/2154 - Train Accuracy: 1.0000, Validation Accuracy: 0.9993, Loss: 0.0060
Epoch   8 Batch 1340/2154 - Train Accuracy: 0.9852, Validation Accuracy: 0.9993, Loss: 0.0089
Epoch   8 Batch 1350/2154 - Train Accuracy: 0.9906, Validation Accuracy: 0.9886, Loss: 0.0074
Epoch   8 Batch 1360/2154 - Train Accuracy: 1.0000, Validation Accuracy: 0.9886, Loss: 0.0072
Epoch   8 Batch 1370/2154 - Train Accuracy: 0.9852, Validation Accuracy: 0.9801, Loss: 0.0099
Epoch   8 Batch 1380/2154 - Train Accuracy: 0.9938, Validation Accuracy: 0.9837, Loss: 0.0028
Epoch   8 Batch 1390/2154 - Train Accuracy: 0.9963, Validati

Epoch   9 Batch   20/2154 - Train Accuracy: 0.9812, Validation Accuracy: 1.0000, Loss: 0.0064
Epoch   9 Batch   30/2154 - Train Accuracy: 0.9766, Validation Accuracy: 1.0000, Loss: 0.0099
Epoch   9 Batch   40/2154 - Train Accuracy: 1.0000, Validation Accuracy: 1.0000, Loss: 0.0026
Epoch   9 Batch   50/2154 - Train Accuracy: 0.9868, Validation Accuracy: 1.0000, Loss: 0.0142
Epoch   9 Batch   60/2154 - Train Accuracy: 1.0000, Validation Accuracy: 1.0000, Loss: 0.0027
Epoch   9 Batch   70/2154 - Train Accuracy: 1.0000, Validation Accuracy: 1.0000, Loss: 0.0031
Epoch   9 Batch   80/2154 - Train Accuracy: 0.9938, Validation Accuracy: 1.0000, Loss: 0.0032
Epoch   9 Batch   90/2154 - Train Accuracy: 0.9975, Validation Accuracy: 1.0000, Loss: 0.0063
Epoch   9 Batch  100/2154 - Train Accuracy: 0.9740, Validation Accuracy: 1.0000, Loss: 0.0178
Epoch   9 Batch  110/2154 - Train Accuracy: 0.9891, Validation Accuracy: 1.0000, Loss: 0.0124
Epoch   9 Batch  120/2154 - Train Accuracy: 1.0000, Validati

Epoch   9 Batch  900/2154 - Train Accuracy: 1.0000, Validation Accuracy: 0.9858, Loss: 0.0062
Epoch   9 Batch  910/2154 - Train Accuracy: 0.9984, Validation Accuracy: 0.9943, Loss: 0.0078
Epoch   9 Batch  920/2154 - Train Accuracy: 0.9993, Validation Accuracy: 0.9858, Loss: 0.0067
Epoch   9 Batch  930/2154 - Train Accuracy: 0.9992, Validation Accuracy: 0.9908, Loss: 0.0122
Epoch   9 Batch  940/2154 - Train Accuracy: 0.9961, Validation Accuracy: 0.9908, Loss: 0.0090
Epoch   9 Batch  950/2154 - Train Accuracy: 0.9868, Validation Accuracy: 0.9780, Loss: 0.0049
Epoch   9 Batch  960/2154 - Train Accuracy: 0.9836, Validation Accuracy: 0.9780, Loss: 0.0071
Epoch   9 Batch  970/2154 - Train Accuracy: 0.9938, Validation Accuracy: 0.9766, Loss: 0.0165
Epoch   9 Batch  980/2154 - Train Accuracy: 1.0000, Validation Accuracy: 0.9716, Loss: 0.0054
Epoch   9 Batch  990/2154 - Train Accuracy: 0.9875, Validation Accuracy: 0.9773, Loss: 0.0032
Epoch   9 Batch 1000/2154 - Train Accuracy: 0.9836, Validati

Epoch   9 Batch 1780/2154 - Train Accuracy: 1.0000, Validation Accuracy: 0.9815, Loss: 0.0059
Epoch   9 Batch 1790/2154 - Train Accuracy: 0.9836, Validation Accuracy: 0.9723, Loss: 0.0074
Epoch   9 Batch 1800/2154 - Train Accuracy: 0.9969, Validation Accuracy: 0.9624, Loss: 0.0046
Epoch   9 Batch 1810/2154 - Train Accuracy: 0.9938, Validation Accuracy: 0.9851, Loss: 0.0081
Epoch   9 Batch 1820/2154 - Train Accuracy: 1.0000, Validation Accuracy: 0.9943, Loss: 0.0021
Epoch   9 Batch 1830/2154 - Train Accuracy: 0.9850, Validation Accuracy: 0.9652, Loss: 0.0109
Epoch   9 Batch 1840/2154 - Train Accuracy: 0.9887, Validation Accuracy: 0.9822, Loss: 0.0032
Epoch   9 Batch 1850/2154 - Train Accuracy: 0.9836, Validation Accuracy: 0.9766, Loss: 0.0070
Epoch   9 Batch 1860/2154 - Train Accuracy: 1.0000, Validation Accuracy: 0.9759, Loss: 0.0038
Epoch   9 Batch 1870/2154 - Train Accuracy: 0.9906, Validation Accuracy: 0.9759, Loss: 0.0185
Epoch   9 Batch 1880/2154 - Train Accuracy: 0.9953, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [36]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [37]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [38]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    return_list = []
    lowered = sentence.lower()
    for word in lowered.split():
        if word in vocab_to_int:
            return_list.append(vocab_to_int[word])
        else:
            return_list.append(vocab_to_int['<UNK>'])
    return return_list


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [39]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [126, 82, 205, 193, 218, 108, 98]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [91, 53, 333, 110, 90, 187, 70, 355, 52, 335, 1]
  French Words: préférées moindres pomme mois trop bien gros requins singes la <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.